# The Simpsons

1.  Lees het bestand van de simpsons in (simpsons_nominal.csv)
Deze tabel bevat informatie over 9 karakters uit de Simpsons.
Deze dataset is al omgezet naar nominale meetwaarden.
We gaan onderzoeken of we 'geslacht' kunnen afleiden uit  haarlengte, gewicht en leeftijd

In [2]:
import numpy as np
import pandas as pd
import graphviz
from id3 import Id3Estimator, export_graphviz

simpsons_nominal = pd.read_csv('data/simpsons_nominal.csv')
display(simpsons_nominal)

,haarlengte,gewicht,leeftijd,geslacht
0,<3,>160,30-40,M
1,>5,<=160,30-40,V
2,<3,<=160,<30,M
3,>5,<=160,<30,V
4,3-5,<=160,<30,V
5,<3,>160,>40,M
6,>5,<=160,>40,V
7,>5,>160,30-40,M
8,>5,>160,>40,M


2.  Bereken zelf de entropie van deze tabel. Schrijf ook de formule op.

3.  Bereken de gain voor iedere kolom.  
Kan je ook de gain van de laatste kolom berekenen?
Schrijf ook telkens de formule op?

4.  Bepaal de subtabel waarbij het gewicht altijd <=160 is.

5.  Wat is de entropie van deze subtabel?
En hoe verhoudt zich dat tov. entropie van de originele tabel?

6.  Bepaal de gain voor iedere kolom van deze subtabel.  
Welke kolom kies je dus voor de volgende stap?

7.  Maak de boomstructuur in Python

8. Welk geslacht verwacht je voor een gloednieuw Simpson karakter met volgende eigenschappen?
 * haarlengte = '>5'
 * gewicht = '>160'
 * leeftijd = '30-40'

9. Extra : Probeer nu eens een DecisionTree op te stellen met DecisionTreeClassifier uit scikit-learn.tree